In [2]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data", 
    train=True,
    download=True, 
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data", 
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NerualNetwork(nn.Module):
    def __init__(self) -> None:
        super(NerualNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10), 
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.6%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



119.3%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()
model = NerualNetwork()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1} \n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
----------------------------------
loss: 2.304350  [    0/60000]
loss: 2.289660  [ 6400/60000]
loss: 2.283795  [12800/60000]
loss: 2.281664  [19200/60000]
loss: 2.249765  [25600/60000]
loss: 2.261220  [32000/60000]
loss: 2.245097  [38400/60000]
loss: 2.236363  [44800/60000]
loss: 2.248309  [51200/60000]
loss: 2.191557  [57600/60000]
Test Error: 
 Accuracy: 47.3%, Avg loss: 0.034620 

Epoch 2 
----------------------------------
loss: 2.234188  [    0/60000]
loss: 2.200241  [ 6400/60000]
loss: 2.188319  [12800/60000]
loss: 2.182141  [19200/60000]
loss: 2.112260  [25600/60000]
loss: 2.144295  [32000/60000]
loss: 2.129091  [38400/60000]
loss: 2.114866  [44800/60000]
loss: 2.146340  [51200/60000]
loss: 2.019668  [57600/60000]
Test Error: 
 Accuracy: 48.6%, Avg loss: 0.032251 

Epoch 3 
----------------------------------
loss: 2.127609  [    0/60000]
loss: 2.052320  [ 6400/60000]
loss: 2.032818  [12800/60000]
loss: 2.023985  [19200/60000]
loss: 1.870492  [25600/60000]
loss: 1.973533

In [10]:
torch.save(model.state_dict(), 'data/model_weights.pth')

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [30]:
next(test_dataloader)

TypeError: 'DataLoader' object is not an iterator

In [34]:
linear_layer = nn.Flatten()
print(linear_layer.type)
print(model.type)
type(linear_layer)
type(model)

<bound method Module.type of Flatten(start_dim=1, end_dim=-1)>
<bound method Module.type of NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)>


__main__.NerualNetwork

In [38]:
import torchvision
from torchinfo import summary
resnet151 = torchvision.models.resnet152()
summary(resnet151, input_size=(1,3, 244,244))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Conv2d: 1-1                            [1, 64, 122, 122]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 122, 122]         128
├─ReLU: 1-3                              [1, 64, 122, 122]         --
├─MaxPool2d: 1-4                         [1, 64, 61, 61]           --
├─Sequential: 1-5                        [1, 256, 61, 61]          --
│    └─Bottleneck: 2-1                   [1, 256, 61, 61]          --
│    │    └─Conv2d: 3-1                  [1, 64, 61, 61]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 61, 61]           128
│    │    └─ReLU: 3-3                    [1, 64, 61, 61]           --
│    │    └─Conv2d: 3-4                  [1, 64, 61, 61]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 61, 61]           128
│    │    └─ReLU: 3-6                    [1, 64, 61, 61]           --
│ 